# K-Nearest Neighbors

## Prepare data

In [ ]:

# Import all modules
from common import concatenate_features, extract_features, load_audio
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from time import time

wav_files = list(map(lambda f: 'wavs/' + f, os.listdir('wavs/')))
male_voices, female_voices = load_audio(wav_files)

male_features = extract_features(male_voices)
female_features = extract_features(female_voices)

male_concatenated = concatenate_features(male_features)
female_concatenated = concatenate_features(female_features)

X = np.vstack((male_concatenated, female_concatenated))
y = np.append([15] * len(male_concatenated), [99] * len(female_concatenated))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

## Create classifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

# Train the classifier
start = time()
knn.fit(X_train, y_train)
end = time()
print('Training time: {} seconds'.format(end - start))

# Test the classifier
start = time()
y_pred = knn.predict(X_test)
end = time()
print('Testing time: {} seconds'.format(end - start))

## Confusion matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8,8))
plt.title('Confusion matrix on test data')
sns.heatmap(cm, annot=True, fmt='d', 
            cmap=plt.cm.Blues, cbar=False, annot_kws={'size':14})
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


# Stats

In [ ]:
true_positive = cm[0][0]
false_positive = cm[0][1]
false_negative = cm[1][0]
true_negative = cm[1][1]

accuracy = np.sum(y_pred == y_test) / len(y_test)
sensitivity = true_positive / (false_negative + true_positive)

print("KNN accuracy: ", accuracy)
print("KNN Sensitivity: ", sensitivity)

## Your turn!

In [ ]:
import librosa
from pydub import AudioSegment

user_file = input('Enter the path to the audio file: ')

if not os.path.isfile(user_file):
    print('File not found!')
    # exit without killing kernel
    raise SystemExit

# get file format
file_format = user_file.split('.')[-1]
if file_format != 'wav':
    sound = AudioSegment.from_file(user_file, format=file_format)
    sound.export('user_audio.wav', format='wav')
    user_file = 'user_audio.wav'

audio_file = librosa.load(user_file)

pruebas = np.array([list(audio_file)], dtype=object)

prueba_features = extract_features(pruebas)

prediccion = knn.predict(prueba_features[0])

total = len(prediccion)
hombres = len(prediccion[prediccion == 15])
mujeres = len(prediccion[prediccion == 99])

if hombres > mujeres:
    print('El audio es de un hombre, con un {:.2f}% de confianza'.format(hombres / total * 100))
else:
    print('El audio es de una mujer, con un {:.2f}% de confianza'.format(mujeres / total * 100))